<a href="https://colab.research.google.com/github/beyg1/Q3/blob/main/Advance_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
from typing import Dict
#Dict is used to specify that a variable will hold a dictionary (a collection of key-value pairs).
#Generate Embeddings:
result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content="Pakistan zinda bad we love our country",
    task_type="retrieval_document",
    title="Embedding of single string",
)

#  1 input > 1 vector output
print(str(result["embedding"])[:50],"... TRIMMED]" )

result['embedding'] #all embeddings printed

In [16]:
len(result['embedding']) #embedding model legth

768

In [17]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would a woodchuck chuck?",
        "How does the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of list of strings",
)

# A list of inputs $ A list of vectors output
for v in result["embedding"]:
    print(str(v)[:50], "... TRIMMED ...", len(v))

[-0.036453027, 0.033254996, -0.03970925, -0.002628 ... TRIMMED ... 768
[-0.01591948, 0.032582663, -0.081024624, -0.011298 ... TRIMMED ... 768
[0.00037063024, 0.03763057, -0.122695684, -0.00951 ... TRIMMED ... 768


# Building Vector Stores & Retreival using Chroma DB and **Langchain**

In [ ]:
!pip install -Uq langchain-chroma

In [20]:

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

!pip install -Uq langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",
                                          google_api_key=userdata.get('GOOGLE_API_KEY'))
embeddings.embed_query("What's our Q1 revenue?") #prints embeddings too

In [24]:
from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings)

In [25]:
list(dir(vectorstore)) #checkout vector store options

['_Chroma__ensure_collection',
 '_Chroma__query_collection',
 '_LANGCHAIN_DEFAULT_COLLECTION_NAME',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_chroma_collection',
 '_client',
 '_client_settings',
 '_collection',
 '_collection_metadata',
 '_collection_name',
 '_cosine_relevance_score_fn',
 '_embedding_function',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_persist_directory',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aa

In [27]:
vectorstore.similarity_search("cat")

[Document(id='a76f4809-00c6-4f7e-9fd4-7df5b59cf3e8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='57bcd874-a19a-4238-b2b4-62e0b1fb7bc8', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b37c3cb1-80d9-4ad5-8af3-749b57110d58', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='b2aa3295-1b5e-4b07-b0f7-890118e85510', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [28]:
await vectorstore.asimilarity_search("cat")

[Document(id='a76f4809-00c6-4f7e-9fd4-7df5b59cf3e8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='57bcd874-a19a-4238-b2b4-62e0b1fb7bc8', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b37c3cb1-80d9-4ad5-8af3-749b57110d58', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='b2aa3295-1b5e-4b07-b0f7-890118e85510', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [29]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("cat")


[(Document(id='a76f4809-00c6-4f7e-9fd4-7df5b59cf3e8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.6688324213027954),
 (Document(id='57bcd874-a19a-4238-b2b4-62e0b1fb7bc8', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.9940857887268066),
 (Document(id='b37c3cb1-80d9-4ad5-8af3-749b57110d58', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.103921890258789),
 (Document(id='b2aa3295-1b5e-4b07-b0f7-890118e85510', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.1072404384613037)]

# Retrieval

In [30]:
from langchain_core.runnables import RunnableLambda
#This imports RunnableLambda, which is used to create a callable object from a
#function. This allows you to treat functions as objects that can be passed around and modified.

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # This is the core of the retrieval logic.
#This wraps the similarity_search function in a RunnableLambda object, making it callable
retriever.batch(["shark"]) #K=1 top 1 result

[[Document(id='b37c3cb1-80d9-4ad5-8af3-749b57110d58', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = userdata.get('GOOGLE_API_KEY')
)

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

# RAG

In [41]:
#RAG pipeline as a sequence of operations.
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

#{"context": retriever, "question": RunnablePassthrough()}: This creates a dictionary.

#"context": retriever: This assigns the retriever we defined earlier to the key "context".
#This means the RAG pipeline will use this retriever to fetch relevant context from our vectorstore.

#"question": RunnablePassthrough(): This assigns RunnablePassthrough() to the key "question".
#RunnablePassthrough simply passes the input question through without any modification.
#This means the original user question will be passed along the pipeline.

#| prompt | llm: This part uses the pipe operator (|) to chain operations together.
#It's like creating an assembly line where data flows from one step to the next.

In [47]:
response = rag_chain.invoke("tell about  sharks?")

print(response.content)

The provided text does not contain any information about sharks.
